<center><h4><font face='宋体'>利用随机森林模型预测手机价格<font></h4><center>

1、数据加载

In [8]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import joblib

# 1. 数据加载
train_df=pd.read_csv(r'.\\archive\\train.csv')
test_df=pd.read_csv(r'.\\archive\\test.csv')


In [ ]:
train_df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


2、数据预处理

In [14]:
# 2. 数据预处理
# 训练集处理
X_train = train_df.drop('price_range', axis=1)
y_train = train_df['price_range']

# 测试集处理：移除不影响建模的id列
test_ids = test_df['id']  # 临时保存id列
X_test = test_df.drop('id', axis=1)  # 确保特征与训练集一致

# 标准化处理
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


3、模型训练

In [15]:
# 3. 训练模型
model = RandomForestClassifier(
    n_estimators=150,
    max_depth=12,
    random_state=42,
    n_jobs=-1
)
model.fit(X_train_scaled, y_train)

RandomForestClassifier(max_depth=12, n_estimators=150, n_jobs=-1,
                       random_state=42)

In [17]:
# 生成预测结果
y_pred = model.predict(X_test_scaled)


4、结果导出

In [18]:
# 5. 重构结果数据框架
result_df = pd.DataFrame({
    'id': test_ids,  # 保留原始id
    'predicted_price_range': y_pred
})

# 6. 合并原始测试数据
# 如需保留所有原始字段：
full_result = pd.concat([test_df, result_df['predicted_price_range']], axis=1)

In [ ]:
# 7. 保存结果
result_df.to_csv('prediction_results.csv', index=False)
full_result.to_csv('full_test_with_predictions.csv', index=False)

# 保存模型工具
joblib.dump(model, 'price_model.pkl')
joblib.dump(scaler, 'price_scaler.pkl')

print("执行完成！生成两个文件：")
print("r'D:\\paq\\yuebing.python_work\\python_work\\test_work\\手机价格\\archive\prediction_results.csv(仅含id和预测结果)")
print("r'D:\\paq\\yuebing.python_work\\python_work\\test_work\\手机价格\\archive\\full_test_with_predictions.csv(全字段+预测结果)")